In [ ]:
!pip3 install pygeohash

In [1]:
import pygeohash as pgh
import os
from pyspark.sql.functions import from_json, col, concat
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when
from pyspark.sql.functions import udf, collect_list, concat_ws, lit
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import SparkSession, Row

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'

#pgh.encode(42.6, -5.6, precision=3)


topic_name = 'Main_Stream'
hostip = "192.168.0.130" # change it to your IP


spark = (
    SparkSession.builder
    .master('local[2]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)





In [62]:
from datetime import datetime
from datetime import timedelta

def process_batch(batch_df, batch_id):
    # Convert DataFrame rows to a list of dictionaries
    rows_list = batch_df.collect()
    dict_list = [row.asDict() for row in rows_list]
    dict_list.sort(key = lambda x: x['producer']) # this is done so that climate data is at start of list for efficient comparison 
    
    mongo_client = MongoClient(
            host=f'{hostip}',
            port=27017
        )
    db = mongo_client['fit3182_db']
    collection = "Main_Stream"
    flag = False
    res = []
    new_dict = {}
    
    
    #Further processing or outputting the list of dictionaries
    if dict_list != []:
        
        # update fields in dict_list:
        for dictionary in dict_list:
            if dictionary['producer'] == ' producer1 ': # if climate data
                values = dictionary['others'].split("/")
                dictionary['date'] = dictionary['time']
                dictionary.update({'air_temperature_celcius' : int(values[0])})
                dictionary.update({'relative_humidity' : float(values[1])})
                dictionary.update({'windspeed_knots' : float(values[2])})
                dictionary.update({'max_wind_speed' : float(values[3])})
                dictionary.update({'precipitation ' : values[4]})
                dictionary.update({'GHI_w/m2': int(values[5])}) 
                del dictionary['others']
                del dictionary['time']
                
                if dictionary['air_temperature_celcius'] > 20 and dictionary['GHI_w/m2'] > 180:
                    dictionary['cause_of_event'] = 'natural'
                else:
                    dictionary['cause_of_event'] = 'other'
                    
                
            else: # if hotspot data
                hotspot_values = dictionary['others'].split("/")
                dictionary.update({'confidence' : int(hotspot_values[0])})
                dictionary.update({'surface_temperature_celcius' : int(hotspot_values[1])})
                del dictionary['others']
             
            
        # check if whether there is only climate data in batch if so insert 1 climate data to mongodb
        only_climate_batch = True
        for dictionary in dict_list:
            if dictionary['producer'] != ' producer1 ':
                only_climate_batch = False
                
                
        if only_climate_batch:
            db[collection].insert_one(dict_list[0])
        else:
            # check and merge hotspot reports with similar location (geohash precision 5) and created time (<=10 minutes apart).
            confidence_lst = []
            surface_temperature_celcius_lst = []
            time_lst = []
            hotspot_lst = [j for i,j in enumerate(dict_list) if dict_list[i]['producer'] != ' producer1 ']
            hotspot_lst.sort(key = lambda x: x['precise_geohash'])
            merged_lst = [] 
            dict_list2 = []
            similar_hotspots_indexes = set() 
            k = 0
            match_found = False
            
            if dict_list[len(dict_list) - 1]['producer'] != ' producer1 ':# if not only has climate data:
                temp = []
                start = 0
                for i in range(0, len(hotspot_lst)): # from here gather hotspots with same precise geohash to another list
                    if i + 1 < len(hotspot_lst):
                        a = datetime.strptime(hotspot_lst[i]["time"], "%H:%M:%S") 
                        b = datetime.strptime(hotspot_lst[i + 1]['time'], "%H:%M:%S") 
                    
                    if i + 1 < len(hotspot_lst) and (hotspot_lst[i]['precise_geohash'] == hotspot_lst[i + 1]['precise_geohash'] and abs(a - b) <= timedelta(hours=0, minutes=10)):
                        #a = datetime.strptime(hotspot_lst[i]["time"], "%H:%M:%S") 
                        #b = datetime.strptime(hotspot_lst[i + 1]['time'], "%H:%M:%S") 
                        #if abs(a - b) <= timedelta(hours=0, minutes=10):
                        continue
                            
                    elif i - start == 0: # if no same precise geohash is found
                        start = i + 1
                        continue
                    else: # merge hotspots
                        temp = hotspot_lst[start: i + 1]
                        start = i + 1
                        
                        for j in range(len(temp)):
                            mydict = temp[j]
                            confidence_lst.append(mydict['confidence'])
                            surface_temperature_celcius_lst.append(mydict['surface_temperature_celcius'])
                            timesplit = mydict['time'].split(':') 
                            time_lst.append( int(timesplit[0]) * 60 * 60 + int(timesplit[1]) * 60 + int(timesplit[2]) )
                            
                        
                        merged_hotspot = hotspot_lst[start - 1] 
                        merged_hotspot['merged_confidence'] =  sum(confidence_lst) / len(confidence_lst)
                        merged_hotspot['merged_surface_temperature_celcius'] = sum(surface_temperature_celcius_lst) / len(surface_temperature_celcius_lst)
                        merged_hotspot['merged_time'] = str(timedelta(seconds= sum(time_lst) / len(time_lst)))
                        del merged_hotspot['confidence']
                        del merged_hotspot['surface_temperature_celcius']
                        del merged_hotspot['time']
                        merged_lst.append(merged_hotspot) 
                        surface_temperature_celcius_lst = []
                        confidence_lst = []
                        time_lst = []
                        
                if len(merged_lst) > 0: 
                    check = True
                    for dictionary in dict_list:
                        for hotspot in merged_lst:
                            # check if hotspot from dict_list does not match geohash from merged_lst
                            if dictionary['producer'] != ' producer1 ' and dictionary['precise_geohash'] == hotspot['precise_geohash']:
                                check = False 
                        if check: # if check is true means either climate data or hotspot with no duplicate geohash, so add to new dictionary
                            dict_list2.append(dictionary)
                        check = True

                    dict_list2.extend(merged_lst)
                    dict_list = dict_list2

                        
                        
                        
                    """
                    if dict_list[i]['producer'] == ' producer1 ':
                        continue
                        
                    mydict = dict_list[i]
                    confidence_lst.append(mydict['confidence'])
                    surface_temperature_celcius_lst.append(mydict['surface_temperature_celcius'])
                    for j in range(i - 1, -1, -1):
                        if dict_list[j]['producer'] != ' producer1 ':
                            x = datetime.strptime(mydict["time"], "%H:%M:%S") 
                            y = datetime.strptime(dict_list[j]["time"], "%H:%M:%S")
                            if mydict['precise_geohash'] == dict_list[j]['precise_geohash'] and abs(x - y) <= timedelta(hours=0, minutes=10):
                                similar_hotspots_indexes.add(i) # indexes added will be used to delete hotspots with similar geohash
                                similar_hotspots_indexes.add(j) # indexes added will be used to delete hotspots with similar geohash
                                confidence_lst.append(dict_list[j]['confidence'])
                                surface_temperature_celcius_lst.append(dict_list[j]['surface_temperature_celcius'])
                                temp = dict_list[j]['time'].split(':') 
                                time_lst.append( int(temp[0]) * 60 * 60 + int(temp[1]) * 60 + int(temp[2]) )
                                match_found = True

                    if match_found:
                        # update the last hotspot data in list by averaging the hotspot with identical geohash
                        mydict['merged_confidence'] = sum(confidence_lst) / len(confidence_lst) 
                        mydict['merged_surface_temperature_celcius_lst'] = sum(surface_temperature_celcius_lst) / len(surface_temperature_celcius_lst)
                        # convert total seconds in timelst to HH:MM:SS then average
                        mydict['merged_time'] = str(timedelta(seconds= sum(time_lst) / len(time_lst)))

                        del mydict['confidence']
                        del mydict['surface_temperature_celcius_lst']
                        del mydict['time']
                        # delete all other hotspots except last after merging
                        while dict_lst != [] and k < len(dict_lst) - 1:
                            if dict_list[k]['producer'] != ' producer1 ':
                                dict_lst.pop(k)
                                continue
                            else:
                                k = k + 1
                                    
                    """



            # Group the Climate and Hotspots streams based on location
            for index, dictionary1 in enumerate(dict_list):
                if dictionary1['producer'] == ' producer1 ':
                    # check geohash for climate stream
                    climate_geohash = dictionary1['geohash']
                    for i in range(index + 1, len(dict_list)):
                        if dict_list[i]['producer'] != ' producer1 ': # if there exist hotspot records

                            # check for geohash for hotspot streams
                            hotspot_geohash = dict_list[i]['geohash']
                            if climate_geohash == hotspot_geohash:
                                res.append(dict_list[i])


                    dictionary1.update({'hotspot': res}) 
                    db = client.fit3182_db

                    if res != []: #if there is at least 1 hotspot date with equal geohash value as 1 climate date
                        db[collection].insert_one(dictionary1)
                        flag = True

                if flag == True: #to ensure not more than 1 climate report is inserted
                    flag = False
                    break 

                res = []



                        




topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', f'{hostip}:9092')
    .option('subscribe', topic_name)
    #.option("startingOffsets", "earliest")
    .option("minOffsetsPerTrigger",1) 
    .option("maxOffsetsPerTrigger",50) 
    .load()
)

topic_stream_df.printSchema()

udf1 = udf(lambda x,y: pgh.encode(x,y,precision=3))
udf2 = udf(lambda x,y: pgh.encode(x,y,precision=5))

topic_stream_df = topic_stream_df.select(split(topic_stream_df.value, ",").alias('data'))
topic_stream_df = topic_stream_df.select( 
                                         element_at('data', 1).alias('time'), 
                                         element_at('data', 2).alias('producer'),
                                         element_at('data', 3).alias('latitude'),
                                         element_at('data', 4).alias('longitude'),
                                         element_at('data', 5).alias('others'),
                                         #Row(key= element_at('data', 3), value= element_at('data', 4))"
                                       )



output_stream_df = (
    topic_stream_df
    #.withColumn('result',  concat(*col_list)) 
    #.withColumn('result',  Row(key= topic_stream_df["latitude"], value= topic_stream_df["longitude"])) 
    .withColumn('latitude', topic_stream_df["latitude"].cast(DoubleType())) 
    .withColumn('longitude', topic_stream_df["longitude"].cast(DoubleType()))
    .withColumn('geohash', udf1('latitude','longitude')) 
    .withColumn('precise_geohash', udf2('latitude','longitude')) 
    
    
    #.groupBy('geohash')  # uncomment this if want to output in "complete" mode
    
    
    #.agg(concat_ws(",", collect_list('latitude')))
    
    #.agg( concat_list('latitude', 'longitude'))
    #.agg(concat_ws(", ", "latitude", "longitude") )
)


console_logger = (
    output_stream_df
    .writeStream
    .outputMode('append')
    .trigger(processingTime = '10 seconds')
    .foreachBatch(process_batch)
    
)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [56]:
writer = console_logger

try:
    query = writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopped query


In [61]:
# test the new collection created
from pprint import pprint

client = MongoClient('192.168.0.130', 27017) 
result = client.list_database_names()
db = client.fit3182_db


collection = db.Main_Stream

#collection.drop() # for dropping collection

cursor = collection.find({}) # same as "cursor = montours.find()" 
for document in cursor: 
    pprint(document)
    
#output below is tested with trigger = 10 seconds, and timedelta = 20 minutes for hotspot terra data and hotspot aqua data for higher chance of having merged hotspots

{'GHI_w/m2': 154,
 '_id': ObjectId('664b76d2f24b387907167384'),
 'air_temperature_celcius': 18,
 'cause_of_event': 'other',
 'date': '01/04/24',
 'geohash': 'r1m',
 'hotspot': [{'confidence': 71,
              'geohash': 'r1m',
              'latitude': -37.4444,
              'longitude': 142.9091,
              'precise_geohash': 'r1mdg',
              'producer': ' producer2 ',
              'surface_temperature_celcius': 45,
              'time': '16:19:20'},
             {'confidence': 58,
              'geohash': 'r1m',
              'latitude': -37.4661,
              'longitude': 143.2419,
              'precise_geohash': 'r1mfg',
              'producer': ' producer2 ',
              'surface_temperature_celcius': 40,
              'time': '16:29:38'},
             {'confidence': 86,
              'geohash': 'r1m',
              'latitude': -37.7845,
              'longitude': 142.9936,
              'precise_geohash': 'r1m9j',
              'producer': ' producer3 ',
        

In [55]:
db.Main_Stream.drop()